In [118]:
import numpy as np
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

texts_true = [
    "Це просто чудовий фільм! Вражаючі ефекти та виконання акторів.",
    "Дуже цікава книга зі захоплюючим сюжетом та глибокими персонажами.",
    "Завдяки цьому продукту я відчуваю справжню радість і задоволення."
]

texts_false = [
    "Фільм був дуже нудним і безцільним. Витрачений час і гроші.",
    "Книга не варта уваги. Плоскі персонажі та слабкий сюжет.",
    "Цей продукт викликає лише розчарування та незадоволення."
]

texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

maxWordsCount = 1000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=maxWordsCount, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_sequence_length)
print(data_pad)

X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indices = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indices]

inputs = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(maxWordsCount, 128)(inputs)
gru_layer1 = LSTM(128, activation='relu', return_sequences=True)(embedding_layer)
gru_layer2 = LSTM(64, activation='relu', return_sequences=True)(gru_layer1)
gru_layer3 = LSTM(32, activation='relu')(gru_layer2)
outputs = Dense(2, activation='softmax')(gru_layer3)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=SGD(learning_rate=0.01))

history = model.fit(X, Y, batch_size=32, epochs=50)

3 3 6
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  7  8  3  9
  10  1 11 12]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4 13  5 14 15
  16  1 17 18]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 19 20 21 22 23
  24 25  2 26]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

Model: "functional_52"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_49 (InputLayer)     │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_52 (Embedding)        │ (None, 100, 128)       │       128,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_111 (LSTM)                 │ (None, 100, 128)       │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_112 (LSTM)                 │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_113 (LSTM)                 │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321,474 (1.23 MB)

 Trainable params: 321,474 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.6933
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5000 - loss: 0.6933
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.5000 - loss: 0.6932
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5000 - loss: 0.6932
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5000 - loss: 0.6932
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5000 - loss: 0.6932
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5000 - loss: 0.6931
Epo

In [119]:
positive_text = "Ця книга мене захопила і принесла велике задоволення."
positive_data = tokenizer.texts_to_sequences([positive_text])
positive_data_pad = pad_sequences(positive_data, maxlen=max_sequence_length)
prediction_positive = model.predict(positive_data_pad)
print(positive_text)
if prediction_positive[0][0] > prediction_positive[0][1]:
    print("Текст позитивний.")
else:
    print("Текст негативний.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 406ms/step
Ця книга мене захопила і принесла велике задоволення.
Текст позитивний.


In [120]:
negative_text = "Цей фільм викликав в мене розчарування та нудьгу."
negative_data = tokenizer.texts_to_sequences([negative_text])
negative_data_pad = pad_sequences(negative_data, maxlen=max_sequence_length)
prediction_negative = model.predict(negative_data_pad)
print(negative_text)
if prediction_negative[0][0] > prediction_negative[0][1]:
    print("Текст негативний.")
else:
    print("Текст позитивний.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Цей фільм викликав в мене розчарування та нудьгу.
Текст негативний.
